In [1]:
#install Kaggle Command Line Tool to use on Jupyter Notebook -here-
%pip install kaggle

Note: you may need to restart the kernel to use updated packages.


In [2]:
#verify it works
!kaggle --version 

Kaggle API 1.6.14


In [3]:
#check commands you can use
!kaggle --help

usage: kaggle [-h] [-v]
              {competitions,c,datasets,d,kernels,k,models,m,files,f,config}
              ...

options:
  -h, --help            show this help message and exit
  -v, --version         show program's version number and exit

commands:
  {competitions,c,datasets,d,kernels,k,models,m,files,f,config}
                        Use one of:
                        competitions {list, files, download, submit, submissions, leaderboard}
                        datasets {list, files, download, create, version, init, metadata, status}
                        kernels {list, files, init, push, pull, output, status}
                        models {instances, get, list, init, create, delete, update}
                        models instances {versions, get, files, init, create, delete, update}
                        models instances versions {init, create, download, delete, files}
                        config {view, set, unset}
    competitions (c)    Commands related to Kaggle 

In [4]:
#view datasets comman
!kaggle datasets -h

usage: kaggle datasets [-h]
                       {list,files,download,create,version,init,metadata,status}
                       ...

options:
  -h, --help            show this help message and exit

commands:
  {list,files,download,create,version,init,metadata,status}
    list                List available datasets
    files               List dataset files
    download            Download dataset files
    create              Create a new dataset
    version             Create a new dataset version
    init                Initialize metadata file for dataset creation
    metadata            Download metadata about a dataset
    status              Get the creation status for a dataset


In [5]:
#list available commands to search for dataset
!kaggle datasets list -h


usage: kaggle datasets list [-h] [--sort-by SORT_BY] [--size SIZE]
                            [--file-type FILE_TYPE] [--license LICENSE_NAME]
                            [--tags TAG_IDS] [-s SEARCH] [-m] [--user USER]
                            [-p PAGE] [-v] [--max-size MAX_SIZE]
                            [--min-size MIN_SIZE]

options:
  -h, --help            show this help message and exit
  --sort-by SORT_BY     Sort list results. Default is 'hottest'. Valid options are 'hottest', 'votes', 'updated', and 'active'
  --size SIZE           DEPRECATED. Please use --max-size and --min-size to filter dataset sizes.
  --file-type FILE_TYPE
                        Search for datasets with a specific file type. Default is 'all'. Valid options are 'all', 'csv', 'sqlite', 'json', and 'bigQuery'. Please note that bigQuery datasets cannot be downloaded
  --license LICENSE_NAME
                        Search for datasets with a specific license. Default is 'all'. Valid options are 'all', 'c

In [6]:
# Search datasets
!kaggle datasets list

401 - Unauthorized - Unauthenticated


In [7]:
# Search datasets for aqi or air quality
!kaggle datasets list -s aqi

401 - Unauthorized - Unauthenticated


In [8]:
#use !kaggle datasets files <owner>/<dataset-name> 

!kaggle datasets files azminetoushikwasi/aqi-air-quality-index-scheduled-daily-update

name            size  creationDate         
-------------  -----  -------------------  
data_date.csv  427KB  2024-06-06 08:52:50  


In [9]:
#look at download options 
!kaggle datasets download -h

usage: kaggle datasets download [-h] [-f FILE_NAME] [-p PATH] [-w] [--unzip]
                                [-o] [-q]
                                [dataset]

options:
  -h, --help            show this help message and exit
  dataset               Dataset URL suffix in format <owner>/<dataset-name> (use "kaggle datasets list" to show options)
  -f FILE_NAME, --file FILE_NAME
                        File name, all files downloaded if not provided
                        (use "kaggle datasets files -d <dataset>" to show options)
  -p PATH, --path PATH  Folder where file(s) will be downloaded, defaults to current working directory
  -w, --wp              Download files to current working path
  --unzip               Unzip the downloaded file. Will delete the zip file when completed.
  -o, --force           Skip check whether local version of file is up to date, force file download
  -q, --quiet           Suppress printing information about the upload/download progress


In [10]:
#download specific file
!kaggle datasets download azminetoushikwasi/aqi-air-quality-index-scheduled-daily-update -f data_date.csv 

Dataset URL: https://www.kaggle.com/datasets/azminetoushikwasi/aqi-air-quality-index-scheduled-daily-update
License(s): ODbL-1.0
... resuming from 428160 bytes (9160 bytes left) ...
 98%|██████████████████████████████████████████████▉ | 418k/427k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 427k/427k [00:00<00:00, 5.73MB/s]


In [11]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
aqi = pd.read_csv("data_date.csv")
aqi


,Date,Country,Status,AQI Value
0,2022-07-21,Albania,Good,14
1,2022-07-21,Algeria,Moderate,65
2,2022-07-21,Andorra,Moderate,55
3,2022-07-21,Angola,Unhealthy for Sensitive Groups,113
4,2022-07-21,Argentina,Moderate,63
...,...,...,...,...
13247,2024-06-06,Uzbekistan,Good,21
13248,2024-06-06,Vatican,Good,25
13249,2024-06-06,Venezuela,Good,10
13250,2024-06-06,Vietnam,Moderate,63


In [13]:
#install sql
import sqlite3

In [14]:
#create database called "aqi.db"
db_connect = sqlite3.connect('aqi.db')

In [15]:
#cur (cursors) allow us the execute sql queries! 
cur = db_connect.cursor()

In [16]:
#create table "aqitable" with composite unique constraint
cur.execute("""
    CREATE TABLE IF NOT EXISTS aqitable (
        ID INTEGER PRIMARY KEY AUTOINCREMENT,
        Date DATETIME NOT NULL,
        Country TEXT,
        Status TEXT,
        AQI_Value INTEGER,
        UNIQUE(Date, Country)
    )
""")


In [17]:
#show all new tables in new SQL database 
cur.execute ("SELECT name from sqlite_master where type='table' order by name;")
results = cur.fetchall()
print(results) 

[('aqitable',), ('sqlite_sequence',)]


In [18]:
#pandas allow to run SQL querry and save data 

import pandas as pd

In [19]:
#Put AQI dataset we found into our sql data as aqi 
aqi.to_sql('aqitable', db_connect, if_exists='replace', index = False)

13252

In [20]:
#query sql tabke inty pythin pandas dataframe
latest_aqi = pd.read_sql_query('select * from aqitable limit 100', db_connect)

In [21]:
#view latest aqi data
latest_aqi

,Date,Country,Status,AQI Value
0,2022-07-21,Albania,Good,14
1,2022-07-21,Algeria,Moderate,65
2,2022-07-21,Andorra,Moderate,55
3,2022-07-21,Angola,Unhealthy for Sensitive Groups,113
4,2022-07-21,Argentina,Moderate,63
...,...,...,...,...
95,2022-07-21,Netherlands,Moderate,70
96,2022-07-21,New Caledonia,Good,15
97,2022-07-21,New Zealand,Good,18
98,2022-07-21,Nigeria,Unhealthy for Sensitive Groups,105


In [22]:
import pandas as pd

# Define the correct paths for the CSV files
aqi_file_path = '/Users/bellewills/data/data_date.csv'
weather_file_path = '/Users/bellewills/data/GlobalWeatherRepository.csv'
pollution_file_path = '/Users/bellewills/data/global air pollution dataset.csv'

# Load datasets into pandas DataFrames
df_aqi = pd.read_csv(aqi_file_path)
df_weather = pd.read_csv(weather_file_path)
df_pollution = pd.read_csv(pollution_file_path)

# Print column names to identify correct names
print("AQI DataFrame columns:", df_aqi.columns)
print("Weather DataFrame columns:", df_weather.columns)
print("Pollution DataFrame columns:", df_pollution.columns)


AQI DataFrame columns: Index(['Date', 'Country', 'Status', 'AQI Value'], dtype='object')
Weather DataFrame columns: Index(['country', 'location_name', 'latitude', 'longitude', 'timezone',
       'last_updated_epoch', 'last_updated', 'temperature_celsius',
       'temperature_fahrenheit', 'condition_text', 'wind_mph', 'wind_kph',
       'wind_degree', 'wind_direction', 'pressure_mb', 'pressure_in',
       'precip_mm', 'precip_in', 'humidity', 'cloud', 'feels_like_celsius',
       'feels_like_fahrenheit', 'visibility_km', 'visibility_miles',
       'uv_index', 'gust_mph', 'gust_kph', 'air_quality_Carbon_Monoxide',
       'air_quality_Ozone', 'air_quality_Nitrogen_dioxide',
       'air_quality_Sulphur_dioxide', 'air_quality_PM2.5', 'air_quality_PM10',
       'air_quality_us-epa-index', 'air_quality_gb-defra-index', 'sunrise',
       'sunset', 'moonrise', 'moonset', 'moon_phase', 'moon_illumination'],
      dtype='object')
Pollution DataFrame columns: Index(['Country', 'City', 'AQI Value',

In [1]:
import sqlite3
import pandas as pd

# Paths to your CSV files
aqi_file_path = '/Users/bellewills/data/data_date.csv'
weather_file_path = '/Users/bellewills/data/GlobalWeatherRepository.csv'
pollution_file_path = '/Users/bellewills/data/global air pollution dataset.csv'

# Load datasets into pandas DataFrames
df_aqi = pd.read_csv(aqi_file_path)
df_weather = pd.read_csv(weather_file_path)
df_pollution = pd.read_csv(pollution_file_path)

# Display the loaded data
print("AQI DataFrame:")
print(df_aqi.head())
print("\nWeather DataFrame:")
print(df_weather.head())
print("\nPollution DataFrame:")
print(df_pollution.head())

# Create a SQLite database
conn = sqlite3.connect('aqi_database2.db')

# Function to list tables in the database
def list_tables(conn):
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [row[0] for row in cursor.fetchall()]
    return tables

# Create and populate the locations table
df_locations_aqi = df_aqi[['Country']].drop_duplicates().reset_index(drop=True)
df_locations_weather = df_weather[['country', 'location_name', 'latitude', 'longitude']].drop_duplicates().reset_index(drop=True)
df_locations_pollution = df_pollution[['Country', 'City']].drop_duplicates().reset_index(drop=True)

df_locations_aqi.rename(columns={'Country': 'country'}, inplace=True)
df_locations_weather.rename(columns={'country': 'country', 'location_name': 'city'}, inplace=True)
df_locations_pollution.rename(columns={'Country': 'country', 'City': 'city'}, inplace=True)

# Combine all locations into one DataFrame
df_locations = pd.concat([df_locations_aqi, df_locations_weather[['country', 'city', 'latitude', 'longitude']], df_locations_pollution[['country', 'city']]]).drop_duplicates().reset_index(drop=True)
df_locations['location_id'] = df_locations.index + 1

# Display the locations DataFrame
print("\nLocations DataFrame:")
print(df_locations.head())

# Save locations to the SQL table
df_locations.to_sql('locations', conn, if_exists='replace', index=False)

# Create and populate the parameters table
parameters = set(df_pollution.columns[2:]).union({'AQI', 'Temperature'})
df_parameters = pd.DataFrame(parameters, columns=['parameter'])
df_parameters['parameter_id'] = df_parameters.index + 1

# Remove any null entries
df_parameters = df_parameters.dropna()

# Display the parameters DataFrame
print("\nParameters DataFrame:")
print(df_parameters.head())

df_parameters.to_sql('parameters', conn, if_exists='replace', index=False)

# Create and populate the measurements table
df_measurements_aqi = df_aqi[['Country', 'Date', 'AQI Value']].copy()
df_measurements_aqi.rename(columns={'Country': 'country', 'Date': 'date', 'AQI Value': 'value'}, inplace=True)
df_measurements_aqi['parameter'] = 'AQI'
df_measurements_aqi['source'] = 'aqi'

df_measurements_weather = df_weather[['country', 'location_name', 'temperature_celsius', 'last_updated']].copy()
df_measurements_weather.rename(columns={'country': 'country', 'location_name': 'city', 'temperature_celsius': 'value', 'last_updated': 'date'}, inplace=True)
df_measurements_weather['parameter'] = 'Temperature'
df_measurements_weather['source'] = 'weather'

df_measurements_pollution = df_pollution.melt(id_vars=['Country', 'City'], var_name='parameter', value_name='value').copy()
df_measurements_pollution.rename(columns={'Country': 'country', 'City': 'city'}, inplace=True)
df_measurements_pollution['source'] = 'pollution'

df_measurements = pd.concat([df_measurements_aqi, df_measurements_weather, df_measurements_pollution]).drop_duplicates().reset_index(drop=True)
df_measurements = df_measurements.merge(df_locations[['location_id', 'country', 'city']], on=['country', 'city'])

# Merge parameter_id into measurements
df_measurements = df_measurements.merge(df_parameters[['parameter_id', 'parameter']], on='parameter')

df_measurements['id'] = df_measurements.index + 1  # Add ID column

# Display the measurements DataFrame
print("\nMeasurements DataFrame:")
print(df_measurements.head())

df_measurements.to_sql('measurements', conn, if_exists='replace', index=False)

# Verify the tables in the database
tables = list_tables(conn)
print("\nTables in the database:", tables)

# Display the first few rows of each table
for table in tables:
    query = f"SELECT * FROM {table} LIMIT 5;"
    df = pd.read_sql_query(query, conn)
    print(f"\nPreview of {table} table:")
    print(df)

# Close the database connection
conn.close()


AQI DataFrame:
         Date    Country                          Status  AQI Value
0  2022-07-21    Albania                            Good         14
1  2022-07-21    Algeria                        Moderate         65
2  2022-07-21    Andorra                        Moderate         55
3  2022-07-21     Angola  Unhealthy for Sensitive Groups        113
4  2022-07-21  Argentina                        Moderate         63

Weather DataFrame:
       country     location_name  latitude  longitude        timezone  \
0  Afghanistan             Kabul     34.52      69.18      Asia/Kabul   
1      Albania            Tirana     41.33      19.82   Europe/Tirane   
2      Algeria           Algiers     36.76       3.05  Africa/Algiers   
3      Andorra  Andorra La Vella     42.50       1.52  Europe/Andorra   
4       Angola            Luanda     -8.84      13.23   Africa/Luanda   

   last_updated_epoch      last_updated  temperature_celsius  \
0          1715849100  2024-05-16 13:15               

In [1]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('aqi_database2.db')

# Function to list all tables
def list_tables(conn):
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    result = conn.execute(query).fetchall()
    tables = [row[0] for row in result]
    return tables

# Function to get a preview of a table
def preview_table(conn, table_name, limit=5):
    query = f"SELECT * FROM {table_name} LIMIT {limit};"
    df = pd.read_sql_query(query, conn)
    return df

# List all tables in the database
tables = list_tables(conn)
print("Tables in the database:", tables)

# Preview each table
for table in tables:
    print(f"\nPreview of {table} table:")
    print(preview_table(conn, table))

# Close the connection
conn.close()


Tables in the database: ['sqlite_sequence', 'locations', 'parameters', 'measurements']

Preview of sqlite_sequence table:
Empty DataFrame
Columns: [name, seq]
Index: []

Preview of locations table:
     country  city latitude longitude  location_id
0    Albania  None     None      None            1
1    Algeria  None     None      None            2
2    Andorra  None     None      None            3
3     Angola  None     None      None            4
4  Argentina  None     None      None            5

Preview of parameters table:
            parameter  parameter_id
0       NO2 AQI Value             1
1         Temperature             2
2     PM2.5 AQI Value             3
3  PM2.5 AQI Category             4
4     CO AQI Category             5

Preview of measurements table:
     country        date value parameter source  city  location_id  \
0    Albania  2022-07-21    14       AQI    aqi  None            1   
1    Algeria  2022-07-21    65       AQI    aqi  None            2   
2    And

In [ ]:

# decided not to use aqitable.db - tried using two db together but kept getting database is locked error for aqitable.db 

# import sqlite3
# import pandas as pd
# import time

# # Function to update the measurements table with retry logic and exponential backoff
# def update_measurements(conn_main, row):
#     retries = 5
#     delay = 1  # initial delay in seconds
#     while retries > 0:
#         try:
#             conn_main.execute('''
#                 UPDATE measurements 
#                 SET status = ?, value = ?
#                 WHERE location_id = ? AND date = ?
#             ''', (row['Status'], row['AQI Value'], row['location_id'], row['Date']))
#             conn_main.commit()
#             break
#         except sqlite3.OperationalError as e:
#             if 'database is locked' in str(e):
#                 retries -= 1
#                 time.sleep(delay)
#                 delay *= 2  # exponential backoff - got this part off google to try and help with the database is locked error but didnt work
#                 if retries == 0:
#                     raise
#             else:
#                 raise

# # Connect to the main database and the second database with a timeout
# conn_main = sqlite3.connect('aqi_database.db', timeout=30)
# conn_second = sqlite3.connect('aqi.db')

# # Load data from the second database
# query = "SELECT Date, Country, Status, `AQI Value` FROM aqitable"
# aqi_data = pd.read_sql_query(query, conn_second)

# # Load the location mapping from the main database
# location_mapping = pd.read_sql_query('SELECT location_id, country FROM locations', conn_main)

# # Merge the AQI data with the location mapping
# merged_data = pd.merge(aqi_data, location_mapping, left_on='Country', right_on='country', how='left')

# # Update the measurements table with the status and AQI Value data
# for index, row in merged_data.iterrows():
#     update_measurements(conn_main, row)

# # Close the connections
# conn_main.close()
# conn_second.close()


In [1]:
%pip install flask

Note: you may need to restart the kernel to use updated packages.


In [2]:
!python -m pip install --upgrade pip

In [3]:
%pip install sqlalchemy
%pip install flask_cors

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# START SERVER!!!!

In [ ]:
from flask import Flask, jsonify, request
import pandas as pd
import sqlite3
from sqlalchemy import create_engine, text
from flask_cors import CORS

app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

# SQLAlchemy setup
db_uri = "sqlite:///aqi_database2.db"  # Locked aqi_database.db so had to duplicate and make .2 to access data
engine = create_engine(db_uri)

@app.route("/")
def hello_world():
    """
    Route for the home page.
    """
    return """
        <h1>Air Quality Database</h1>
        <p>Follow the links to view data: <a href="/locations">Locations</a>, <a href="/parameters">Parameters</a>, <a href="/measurements">Measurements</a>.</p>
    """

@app.route("/locations")
def list_locations():
    """
    Route to list all locations, limited to the first 24,000 entries.
    """
    with engine.connect() as connection:
        result = connection.execute(text("SELECT location_id, country, city, latitude, longitude FROM locations LIMIT 24000"))
        output = ["<h1>List of Locations</h1>", "<ul>"]

        for location in result:
            id = location[0]
            country = location[1]
            city = location[2] if location[2] else ''
            latitude = location[3]
            longitude = location[4]
            output.append(f"""
                <li>
                    {country} - {city} (Lat: {latitude}, Long: {longitude}): 
                    <a href="/parameters/{id}">Parameters</a>
                    - <a href="/measurements/{id}">Measurements Data</a>
                </li>"""
            )

        output.append("</ul>")
        return "\n".join(output)

@app.route("/parameters")
def list_parameters():
    """
    Route to list all parameters.
    """
    with engine.connect() as connection:
        result = connection.execute(text("SELECT parameter_id, parameter FROM parameters"))
        output = ["<h1>List of Parameters</h1>", "<ul>"]

        for item in result:
            output.append(f"<li>{item[0]}: {item[1]}</li>")

        output.append("</ul>")
        return "\n".join(output)

@app.route("/measurements")
def list_measurements():
    """
    Route to list all measurements, limited to the first 24,000 entries.
    """
    with engine.connect() as connection:
        result = connection.execute(text(f"""
            SELECT measurements.id, parameters.parameter, measurements.value, measurements.date, measurements.location_id, measurements.parameter_id
            FROM measurements
            JOIN parameters ON measurements.parameter_id = parameters.parameter_id
            LIMIT 24000
        """))
        output = ["<h1>List of Measurements</h1>", "<ul>"]

        for item in result:
            output.append(f"<li>ID: {item[0]}, Parameter: {item[1]}, Value: {item[2]}, Date: {item[3]}, Location ID: {item[4]}, Parameter ID: {item[5]}</li>")

        output.append("</ul>")
        return "\n".join(output)

def get_location_name(session, id):
    """
    Function to get the name of a location.
    """
    result = session.execute(text("SELECT country, city FROM locations WHERE location_id = :id"), {"id": id})
    location = result.first()
    return f"{location[0]} - {location[1]}" if location[1] else location[0]

@app.route("/parameters/<int:location_id>")
def list_parameters_by_location(location_id):
    """
    Route to list parameters for a given location.
    """
    with engine.connect() as connection:
        result = connection.execute(text("SELECT parameter_id, parameter FROM parameters"))
        location_name = get_location_name(connection, location_id)
        output = [
            f"<h1>Parameters in {location_name}</h1>",
            "<ul>"
        ]
        for item in result:
            output.append(f"<li>{item[0]}: {item[1]}</li>")

        output.append("</ul>")
        return "\n".join(output)
# Pput limit of the actual amount of locations and meausrements to make sure all data is being shown as had problems with loading that much data

@app.route("/measurements/<int:location_id>")
def list_measurements_by_location(location_id):
    """
    Route to list measurements for a given location, limited to the first 24,000 entries.
    """
    with engine.connect() as connection:
        result = connection.execute(text(f"""
            SELECT measurements.id, parameters.parameter, measurements.value, measurements.date, measurements.location_id, measurements.parameter_id
            FROM measurements
            JOIN parameters ON measurements.parameter_id = parameters.parameter_id
            WHERE measurements.location_id = :id
            LIMIT 24000
        """), {"id": location_id})
        location_name = get_location_name(connection, location_id)
        output = [
            f"<h1>Measurements Data for {location_name}</h1>",
            "<ul>"
        ]
        for item in result:
            output.append(f"<li>ID: {item[0]}, Parameter: {item[1]}, Value: {item[2]}, Date: {item[3]}, Location ID: {item[4]}, Parameter ID: {item[5]}</li>")

        output.append("</ul>")
        return "\n".join(output)

@app.route("/json/locations")
def json_list_locations():
    """
    Route to list all locations in JSON format, limited to the first 24,000 entries.
    """
    with engine.connect() as connection:
        result = connection.execute(text("SELECT location_id, country, city, latitude, longitude FROM locations LIMIT 24000"))
        data = [{'id': location[0], 'country': location[1], 'city': location[2], 'latitude': location[3], 'longitude': location[4]} for location in result]
        return jsonify(data=data)

@app.route("/json/parameters")
def json_list_parameters():
    """
    Route to list all parameters in JSON format.
    """
    with engine.connect() as connection:
        result = connection.execute(text("SELECT parameter_id, parameter FROM parameters"))
        data = [{'id': parameter[0], 'name': parameter[1]} for parameter in result]
        return jsonify(data=data)

@app.route("/json/location/<int:location_id>/measurements")
def json_location_measurements(location_id):
    """
    Route to list all measurements for a given location in JSON format, limited to the first 24,000 entries.
    """
    # Had to limit the amount of entries because all entries were not loading and freezing because there were too many

    with engine.connect() as connection:
        result = connection.execute(text(f"""
            SELECT measurements.id, parameters.parameter, measurements.value, measurements.date, measurements.location_id, measurements.parameter_id
            FROM measurements
            JOIN parameters ON measurements.parameter_id = parameters.parameter_id
            WHERE measurements.location_id = :id
            LIMIT 24000
        """), {"id": location_id})
        data = [{'id': measurement[0], 'parameter': measurement[1], 'value': measurement[2], 'date': measurement[3], 'location_id': measurement[4], 'parameter_id': measurement[5]} for measurement in result]
        return jsonify(data=data)

if __name__ == "__main__":
    app.run(debug=False, port=5101)  # Make sure debug is off!!!!


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5101
Press CTRL+C to quit
127.0.0.1 - - [08/Jun/2024 22:34:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2024 22:34:48] "GET /measurements HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2024 22:35:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2024 22:35:09] "GET /locations HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2024 22:36:48] "GET /json/locations HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2024 22:37:04] "GET /json/location/1/measurements?page=1&per_page=100 HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2024 22:37:13] "GET /json/parameters HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2024 23:04:01] "GET /parameters/1 HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2024 23:04:46] "GET /json/locations HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2024 23:04:59] "GET /json/locations HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2024 23:05:32] "GET /json/location/1/measurements?page=1&per_page=100 HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2024 23:05:50] "GET /json/parameters HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/